<a href="https://colab.research.google.com/github/GoldPapaya/info256-applied-nlp/blob/main/7.lm/HW6_Perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/7.lm/HW6_Perplexity.ipynb)

# HW6: Perplexity

In this homework, you will implement a function to calculate the perplexity of the n-gram language models we covered in lab, and experiment with different sequences to better understand both n-gram LMs as well as the perplexity metric.

In [3]:
import copy
from collections import Counter

import nltk
import numpy as np
from nltk import sent_tokenize, word_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
def read_file(filename):
    sequences = []
    with open(filename) as file:
        data = file.read()
        sents = sent_tokenize(data)
        for sent in sents:
            tokens = word_tokenize(sent)
            sequences.append(tokens)
    return sequences

In [5]:
!wget --no-check-certificate https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt

--2025-10-03 18:03:48--  https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/1342_pride_and_prejudice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691804 (676K) [text/plain]
Saving to: ‘1342_pride_and_prejudice.txt.1’

1342_pride_and_prej 100%[===================>] 675.59K  --.-KB/s    in 0.03s   

2025-10-03 18:03:48 (20.4 MB/s) - ‘1342_pride_and_prejudice.txt.1’ saved [691804/691804]



In [6]:
# Read data from file and tokenize them into sequences comprised of tokens.

# Pride and Prejudice (Jane Austen)
sequences = read_file("1342_pride_and_prejudice.txt")

max_sequences = 10000

In [7]:
class NgramModel():

    def __init__(self, sequences, order):

        # For this exercise we're going to encode the LM as a sparse dictionary (trading less storage for more compute)
        # We'll store the LM as a dictionary with the conditioning context as keys; each value is a
        # Counter object that keeps track of the number of times we see a word following that context.
        self.counts = {}

        # Markov order (order 1 = conditioning on previous 1 word; order 2 = previous 2 words, etc.)
        self.order = order

        vocab = {"[END]": 0}

        for s_idx, tokens in enumerate(sequences):
            # We'll add [START] and [END] tokens to encode the beginning/end of sentences
            tokens = ["[START]"] * order + tokens + ["[END]"]

            if s_idx == 0:
                print(tokens)

            for i in range(order, len(tokens)):
                context = " ".join(tokens[i - order:i])
                word = tokens[i]

                if word not in vocab:
                    vocab[word] = len(vocab)

                # For just the first sentence, print the conditioning context + word
                if s_idx == 0:
                    print("Context: %s Next: %s" % (context.ljust(50), word))

                if context not in self.counts:
                    self.counts[context] = Counter()
                self.counts[context][word] += 1



    def sample(self, context):
        total = sum(self.counts[context].values())

        dist = []
        vocab = []

        # Create a probability distribution for each conditioning context, over the vocab that we've observed it with.
        for idx, word in enumerate(self.counts[context]):
            prob = self.counts[context][word]/total
            dist.append(prob)
            vocab.append(word)

        index = np.argmax(np.random.multinomial(1, pvals=dist))
        return vocab[index]

    def generate_sequence(self, keep_ends=True):
        generated = ["[START]"] * (self.order)
        word = None
        while word != "[END]":
            context = ' '.join(generated[-self.order:] if self.order > 0 else "")
            word = self.sample(context)
            generated.append(word)
        if not keep_ends:
            generated = generated[self.order:-1]
        return " ".join(generated)



Let's create some language models of different orders from *Pride and Prejudice*.

In [8]:
ngram0 = NgramModel(sequences[:max_sequences], order=0)

['Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune', ',', 'must', 'be', 'in', 'want', 'of', 'a', 'wife', '.', '[END]']
Context:                                                    Next: Chapter
Context:                                                    Next: 1
Context:                                                    Next: It
Context:                                                    Next: is
Context:                                                    Next: a
Context:                                                    Next: truth
Context:                                                    Next: universally
Context:                                                    Next: acknowledged
Context:                                                    Next: ,
Context:                                                    Next: that
Context:                                                   

In [9]:
ngram1 = NgramModel(sequences[:max_sequences], order=1)

['[START]', 'Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune', ',', 'must', 'be', 'in', 'want', 'of', 'a', 'wife', '.', '[END]']
Context: [START]                                            Next: Chapter
Context: Chapter                                            Next: 1
Context: 1                                                  Next: It
Context: It                                                 Next: is
Context: is                                                 Next: a
Context: a                                                  Next: truth
Context: truth                                              Next: universally
Context: universally                                        Next: acknowledged
Context: acknowledged                                       Next: ,
Context: ,                                                  Next: that
Context: that                                   

In [10]:
ngram2 = NgramModel(sequences[:max_sequences], order=2)

['[START]', '[START]', 'Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that', 'a', 'single', 'man', 'in', 'possession', 'of', 'a', 'good', 'fortune', ',', 'must', 'be', 'in', 'want', 'of', 'a', 'wife', '.', '[END]']
Context: [START] [START]                                    Next: Chapter
Context: [START] Chapter                                    Next: 1
Context: Chapter 1                                          Next: It
Context: 1 It                                               Next: is
Context: It is                                              Next: a
Context: is a                                               Next: truth
Context: a truth                                            Next: universally
Context: truth universally                                  Next: acknowledged
Context: universally acknowledged                           Next: ,
Context: acknowledged ,                                     Next: that
Context: , that                      

**Q1.** Create a `perplexity` function that can takes two arguments: a.) a model of *any* ngram order (from the class above); and b.) a sequence to calculate perplexity for.  You'll recall from class that perplexity under a particular language model for sequence $w$ is given by the following equation:

$$
\textrm{perplexity}_{model}(w) = \exp\left(-{1 \over N} \sum_{i=1}^N \log P_{model}(w_i) \right)
$$

$P_{model}(w_i)$ calculates the probability of token $w_i$ using whatever assumptions that model makes -- for a bigram model (order 1), this is $P(w_i \mid w_{i-1})$, for a trigram model (order 2), this is $P(w_i \mid w_{i-2}, w_{i-1})$, etc.  Two things to note:

* When calculating the probability of the first word(s), be sure to get the conditioning context right.  The conditioning context for the first word in a trigram model, for example, is $P(w_i \mid$ [START] [START]$)$.
* Perplexity is only calculated for the words in the actual sequence.  We don't include the $P$([START]) or $P$([END]) in the perplexity calculuation.


*Hint*: when working on this function, you might want to debug by printing out the probabilities of each $w_i$.

In [26]:
from math import log, exp

def perplexity(model, tokens):
    n = len(tokens)
    start_padding = ["[START]"] * model.order
    end_padding = ["[END]"]
    tokens = start_padding + tokens + end_padding
    p_sum = 0.0
    for i in range(model.order, len(tokens) - 1): # all tokens excluding start and end
        context = " ".join(tokens[i - model.order:i]) if model.order > 0 else ""
        word = tokens[i]

        # calc P(word|context)
        if context in model.counts and word in model.counts[context]:
            total = sum(model.counts[context].values())
            prob = model.counts[context][word] / total
        else:
            prob = 1e-10  # assign small probability to avoid 0

        p_sum += log(prob)
    return exp(-1/n * p_sum)

**Q2**. Execute that perplexity function for the following language models:

In [27]:
perplexity(ngram0, word_tokenize("She was a great friend of Mr. Bingley."))

202.10235240923632

In [28]:
perplexity(ngram1, word_tokenize("She was a great friend of Mr. Bingley."))

23.56747449844477

In [29]:
perplexity(ngram2, word_tokenize("She was a great friend of Mr. Bingley."))

10.416705950936633

**Q3.** What is the perplexity of "She was a really great friend of Mr. Bingley." in the trigram language model trained above?

Explain in 100 words what behavior is expected (and correct) given how an n-gram language model works and the data we are training it on.

In [30]:
perplexity(ngram2, word_tokenize("She was a really great friend of Mr. Bingley."))

4044.792933650393

We expect a higher perplexity value

**Q4.** What 1-token sequence yields the lowest perplexity for the 0-order n-gram model? Why?

In [ ]:
perplexity(ngram0, word_tokenize("YOUR_ANSWER_HERE"))

**Q5.** Write a function to find the n-token sequence with the lowest perplexity given an n-gram model. Explain why it should work.

In [37]:
def min_ppl_sequence(model, n=5):
    """Return an n-token long string that yields the lowest possible perplexity under the provided model."""
    seq = ["[START]"] * model.order  # begin with start tokens
    generated = []

    for _ in range(n):
        context = " ".join(seq[-model.order:] if model.order > 0 else "")

        # get the most frequent next word in this context
        if context in model.counts:
            next_word = model.counts[context].most_common(1)[0][0]
        else:
            # unseen context fallback
            next_word = "[END]"

        seq.append(next_word)
        generated.append(next_word)

    return " ".join(generated)


In [38]:
min_ppl_sequence(ngram1, n=3)

'“ I am'

In [39]:
# the provided call needs the tokens from above or it wont work, not the whole phrase.
from nltk import word_tokenize

tokens = word_tokenize(min_ppl_sequence(ngram1, n=3))
perplexity(ngram1, tokens)

7.761070591311059

In [36]:
perplexity(ngram1, word_tokenize("She was a great friend of Mr. Bingley."))

10000000000.000004